In [1]:
import numpy as np 
import pandas as pd 
import multiprocessing as multi
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import time

In [2]:
%%capture PC_name 
!hostname

**Setting notebook parameters**

In [3]:
#setting dots per inch for images
dpi = 90

#setting number of processors for multiprocessing
PC_name_str = PC_name.stdout[0:-2]

if (PC_name_str == 'arc10'or PC_name_str == 'science11' or 
        PC_name_str == 'science10'):
    cores = 42        
#     cores = multi.cpu_count()
    
else:
    cores = multi.cpu_count()

print(PC_name_str)
print(cores)

arc10
42


In [4]:
df_sdss_gals = pd.read_csv('../codes_for_SDSS/Data/SAS_FITS/galaxy_DR12v5_CMASSLOWZTOT_all_sky_Comov.csv',
                         index_col = 0)
df_sdss_gals.describe().apply(lambda s: s.apply('{0:.3f}'.format))

/vol/arc10/data1/amirza/Misc/anaconda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,RA,DEC,Z,r_comov[Mpc],X_comov[Mpc],Y_comov[Mpc],Z_comov[Mpc]
count,1325735.000,1325735.000,1325735.000,1325735.000,1325735.000,1325735.000,1325735.000
mean,178.676,23.417,0.452,1757.113,-402.293,-50.546,660.839
std,92.194,18.180,0.157,557.464,1329.136,855.185,546.961
min,0.000,-10.997,0.000,0.022,-3378.717,-2920.029,-544.979
25%,135.845,8.361,0.347,1409.890,-1386.732,-665.693,208.113
50%,181.891,21.477,0.486,1900.086,-832.396,-22.369,573.208
75%,227.366,35.940,0.563,2155.089,796.933,565.573,1039.145
max,359.999,68.738,1.137,3721.922,3317.237,2869.830,3005.346


In [5]:
df_act_clus = pd.read_csv('Data/ACT_DR5_Clusters_Comov.csv', index_col = 0)
df_act_clus.describe()

,RADeg,decDeg,redshift,M500,r_comov,x_comov,y_comov,z_comov
count,4195.000000,4195.000000,4195.000000,4195.000000,4195.000000,4195.000000,4195.000000,4195.000000
mean,151.193567,-11.682177,0.550965,2.621147,2050.633192,318.987671,219.616443,-382.659816
std,117.664528,24.306970,0.260175,1.092935,819.352266,1599.680360,1106.626053,892.947405
min,0.020896,-59.987040,0.035200,1.042072,154.676621,-4170.040899,-3524.075869,-3823.141694
25%,39.386235,-34.828811,0.354170,1.961358,1434.406287,-1091.760851,-524.758254,-984.745914
50%,144.342767,-2.252158,0.519816,2.361268,2012.598290,723.644358,259.891557,-67.613160
75%,232.604297,8.957936,0.709455,2.937229,2607.246878,1517.502717,992.884047,248.498785
max,359.988280,20.486048,1.910000,13.442958,5173.142447,4421.617253,3457.028008,1366.557544


In [26]:
def delta_gal_mean_func(df_act_clus_available, cell_size, radius):
    
    delta_gal_cube = np.zeros(len(df_act_clus_available))
    
    for i, clus in df_act_clus_available.iterrows():
        df_gals_around = df_sdss_gals[(df_sdss_gals['X_comov[Mpc]'] < (clus['x_comov'] + radius)) &
                                      (df_sdss_gals['X_comov[Mpc]'] > (clus['x_comov'] - radius)) &
                                      (df_sdss_gals['Y_comov[Mpc]'] < (clus['y_comov'] + radius)) &
                                      (df_sdss_gals['Y_comov[Mpc]'] > (clus['y_comov'] - radius)) &
                                      (df_sdss_gals['Z_comov[Mpc]'] < (clus['z_comov'] + radius)) &
                                      (df_sdss_gals['Z_comov[Mpc]'] > (clus['z_comov'] - radius))].copy()
        
        
        #moving the galxies cube to lie within 0 to 240,000 h^-1 kpc 
        df_gals_around['X_comov[Mpc]'] -= (clus['x_comov'] - radius)
        df_gals_around['Y_comov[Mpc]'] -= (clus['y_comov'] - radius)
        df_gals_around['Z_comov[Mpc]'] -= (clus['z_comov'] - radius)

        df_gals_around['X_comov[Mpc]'] = df_gals_around['X_comov[Mpc]'] / cell_size
        df_gals_around['Y_comov[Mpc]'] = df_gals_around['Y_comov[Mpc]'] / cell_size
        df_gals_around['Z_comov[Mpc]'] = df_gals_around['Z_comov[Mpc]'] / cell_size

        #making tuples, converting tuples to cell coordinates
        df_gals_around["cell"] = list(zip(df_gals_around['X_comov[Mpc]'].astype(int), 
                                          df_gals_around['Y_comov[Mpc]'].astype(int), 
                                          df_gals_around['Z_comov[Mpc]'].astype(int)))
        
        diameter = radius * 2
        #array to store number of galaxies in the cells
        gals_in_cell = np.zeros((diameter//cell_size, diameter//cell_size, diameter//cell_size))

        #counting number of galaxies in the cells
        for cell in df_gals_around["cell"]:
            x, y, z = cell
    #         gals_in_cell[x, y, z] += 1

            try:
                gals_in_cell[x, y, z] += 1

            except:
                print('\n')
                print(i)
                print(gals_in_cell.shape)
                print(x, y, z)
                print(df_act_clus_available.loc[[i]])
                raise SystemExit

        delta_gal_cube[i] = np.mean(gals_in_cell)
    
    return np.mean(delta_gal_cube)

In [17]:
def overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, clus_cube_size, df_gal_padded):

    #converting strings into floats
    clus_x = float(clus_x); clus_y = float(clus_y); clus_z = float(clus_z)
    
    #filtering galaxies in a cube of 240,000 h^-1 kpc around the given cluster
    df_gal_select = df_gal_padded[(df_gal_padded['x[kpc/h]'] > (clus_x - clus_cube_size//2)) & 
                                  (df_gal_padded['x[kpc/h]'] < (clus_x + clus_cube_size//2)) & 
                                  (df_gal_padded['y[kpc/h]'] > (clus_y - clus_cube_size//2)) & 
                                  (df_gal_padded['y[kpc/h]'] < (clus_y + clus_cube_size//2)) & 
                                  (df_gal_padded['z[kpc/h]'] > (clus_z - clus_cube_size//2)) & 
                                  (df_gal_padded['z[kpc/h]'] < (clus_z + clus_cube_size//2))]
        
    #making copy to extract coordinates of cells containing the galaxies
    df_gal_cube = df_gal_select.copy()
    
    #moving the galxies cube to lie within 0 to 240,000 h^-1 kpc 
    df_gal_cube['x[kpc/h]'] -= (clus_x - (clus_cube_size//2))
    df_gal_cube['y[kpc/h]'] -= (clus_y - (clus_cube_size//2))
    df_gal_cube['z[kpc/h]'] -= (clus_z - (clus_cube_size//2))

    #dividing by 5000 (integer-div) so we get cell coordinates
    df_gal_cube['x[kpc/h]'] = df_gal_cube['x[kpc/h]'] // cell_size
    df_gal_cube['y[kpc/h]'] = df_gal_cube['y[kpc/h]'] // cell_size
    df_gal_cube['z[kpc/h]'] = df_gal_cube['z[kpc/h]'] // cell_size
    
    #making tuples, converting tuples to cell coordinates
    df_gal_cube["cell"] = list(zip(df_gal_cube['x[kpc/h]'].astype(int), df_gal_cube['y[kpc/h]'].astype(int), df_gal_cube['z[kpc/h]'].astype(int)))
#     df_gal_cube["cell"] = list(zip(df_gal_cube['x[kpc/h]'], df_gal_cube['y[kpc/h]'], df_gal_cube['z[kpc/h]']))
    
    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    
    #counting number of galaxies in the cells
    for i, cell in enumerate(df_gal_cube["cell"]):
        x, y, z = cell
#         gals_in_cell[x, y, z] += 1
        
        try:
            gals_in_cell[x, y, z] += 1
            
        except:
            print('\n')
            print(i)
            print(clus_cube_size)
            print(cell_size)
            print(clus_cube_size//cell_size)
            print(gals_in_cell.shape)
            print(x, y, z)
            print(clus_x, clus_y, clus_z)
            raise SystemExit
            print(df_gal_select.loc[[i]])
            
            
        
    gals_in_cell_up_err, gals_in_cell_low_err = gehrels_confidence_limits(gals_in_cell)
    gals_in_cell_avg_err = (gals_in_cell_low_err + gals_in_cell_up_err) / 2 
    
    #determining the overdensity of galaxies    
    delta_gal = (gals_in_cell/delta_gal_mean) - 1
    
    delta_gal_error = gals_in_cell_avg_err/delta_gal_mean  
    
    #smoothing the overdensity of galaxies
#     delta_gal_smooth = gaussian_filter(delta_gal, sigma = sigma_in_pix)

    delta_gal_smooth = delta_gal.copy() #to avoid error calculation since 
    #smoothing not affecting velocities much (apparently, not at all with 2 Mpc)
        
    #obtaining matter overdensity from galaxies overdensity
    delta_matter = delta_gal_smooth / b
    delta_matter_error = delta_gal_error / b
    
    return delta_matter, delta_matter_error


**Calculating differential, numerator & denominator for Eq. 1**

In [33]:
def vel_terms_calc(cell_size, clus_cube_size):
    
    #calculating the differential in the Eq. 1
    dy_cubed = cell_size**3
    
    #specifing position of the clusters
    Rclus_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_x[:] = clus_cube_size//2
    Rclus_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_y[:] = clus_cube_size//2
    Rclus_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_z[:] = clus_cube_size//2

    #generating meshgrid containing coordinates of the centers of cells
    Rcell_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))

    for i, val in enumerate(range(cell_size//2, clus_cube_size, cell_size)):
        Rcell_x[i,:,:] = val
        Rcell_y[:,i,:] = val
        Rcell_z[:,:,i] = val

    #evaluating the term in the denominator of Eq. 1 of Tanimura et al. 2020
    denom = np.sqrt((Rcell_x - Rclus_x)**2 + (Rcell_y - Rclus_y)**2 + (Rcell_z - Rclus_z)**2)**(3)

    #evaluating the direction term in the numerator of Eq. 1
    numer_x = Rcell_x - Rclus_x
    numer_y = Rcell_y - Rclus_y
    numer_z = Rcell_z - Rclus_z
    
    return (dy_cubed, numer_x, numer_y, numer_z, denom)


**Calculating velocity of clusters according to Eq. 1**

In [31]:
def clus_velocity_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, vel_terms, sigma_in_pix, 
                       clus_cube_size, df_gal_padded):
    
    delta_matter, delta_matter_error = overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, 
                                          clus_cube_size, df_gal_padded)
    
    
    dy_cubed, numer_x, numer_y, numer_z, denom = vel_terms
    
       
    #estimating velocity in x direction
    integrand_x = dy_cubed * delta_matter * (numer_x/(h*1e3)) / denom #in units of Mpc    
    integrand_x_error = dy_cubed * delta_matter_error * (numer_x/(h*1e3)) / denom
    
    dist = np.cbrt(denom)
    integrand_x = integrand_x[dist < (clus_cube_size//2)]
    integrand_x_error = integrand_x_error[dist < (clus_cube_size//2)]
    
    vx_est = pre_fac * np.sum(integrand_x)
    vx_est_error = pre_fac * np.sqrt(np.sum(integrand_x_error**2))
    
        
    #estimating velocity in y direction
    integrand_y = dy_cubed * delta_matter * (numer_y/(h*1e3)) / denom #in units of Mpc    
    integrand_y_error = dy_cubed * delta_matter_error * (numer_y/(h*1e3)) / denom
    
       
    integrand_y = integrand_y[dist < (clus_cube_size//2)]
    integrand_y_error = integrand_y_error[dist < (clus_cube_size//2)]
    
    vy_est = pre_fac * np.sum(integrand_y)
    vy_est_error = pre_fac * np.sqrt(np.sum(integrand_y_error**2))
    
    
    #estimating velocity in z direction
    integrand_z = dy_cubed * delta_matter * (numer_z/(h*1e3)) / denom #in units of Mpc   
    integrand_z_error = dy_cubed * delta_matter_error * (numer_z/(h*1e3)) / denom
    
    integrand_z = integrand_z[dist < (clus_cube_size//2)]
    integrand_z_error = integrand_z_error[dist < (clus_cube_size//2)]
    
    vz_est = pre_fac * np.sum(integrand_z)
    vz_est_error = pre_fac * np.sqrt(np.sum(integrand_z_error**2))
    
    
    return(vx_est, vy_est, vz_est, vx_est_error, vy_est_error, vz_est_error)


In [9]:
def egde_gals_counter(clus, radius):
    
    edge_gals = (df_sdss_gals[(df_sdss_gals['X_comov[Mpc]'] > (clus['x_comov'] + radius))].shape[0],
                 df_sdss_gals[(df_sdss_gals['X_comov[Mpc]'] < (clus['x_comov'] - radius))].shape[0],
                 df_sdss_gals[(df_sdss_gals['Y_comov[Mpc]'] > (clus['y_comov'] + radius))].shape[0],
                 df_sdss_gals[(df_sdss_gals['Y_comov[Mpc]'] < (clus['y_comov'] - radius))].shape[0],
                 df_sdss_gals[(df_sdss_gals['Z_comov[Mpc]'] > (clus['z_comov'] + radius))].shape[0],
                 df_sdss_gals[(df_sdss_gals['Z_comov[Mpc]'] < (clus['z_comov'] - radius))].shape[0])
    
    
    if 0 in edge_gals:
        inside_clus = False
    else:
        inside_clus = True
        
    return (edge_gals, inside_clus)

In [34]:
%%time


b = 2                                #bias for LOWZ & CMASS galaxies as taken by Tanimura et al. 2020

# clus_mass_bins_results = pd.DataFrame(columns=['Cell Size', 'Act Cube Size', 
#                                 'Cube Size Set', 'Clusters', 'Lower lim', 'Upper lim',
#                                 'Slope - Vlos', 'Error - Slope - Vlos', 
#                                 'Intercept - Vlos', 'Scatter - Vlos'])



# clus_cube_sizes = [400000, 500000, 60000]
for radius in [250]:
    for cell_size in [10]:                       #Mpc, size of pixel or cell
        
        t = time.time()              
        
        print(cell_size, radius)
        
        clus_cube_size = int(radius * 2)
        
#         clus_param = [(clus, radius) for i, clus in df_act_clus.iterrows()]
#         pool = multi.Pool(processes = cores)
#         edge_info = pool.starmap(egde_gals_counter, clus_param)
#         inside_clus_list = [i[1] for i in edge_info]        
#         df_act_clus_available = df_act_clus[inside_clus_list].reset_index()
        
        print('Available cluster identified')
    
        
#         delta_gal_mean = delta_gal_mean_func(df_act_clus_available, cell_size, 
#                                              radius)
        print('\n')
        print('Delta_gal: ' + str(delta_gal_mean))
        print('\n')
        
        vel_terms = vel_terms_calc(cell_size, clus_cube_size)
        
        no_of_clus = len(df_act_clus_available)

        clus_param_2 = list(zip(df_act_clus_available['x_comov'], 
                                df_act_clus_available['y_comov'], 
                                df_act_clus_available['z_comov'], 
                                [delta_gal_mean] * no_of_clus, 
                                [cell_size] * no_of_clus,
                                [vel_terms]  * no_of_clus, 
#                                 [sigma_in_pix] * no_of_clus,
                                [clus_cube_size] * no_of_clus
#                                 [df_gal_padded] * no_of_clus
                               ))

#         pool = multi.Pool(processes = cores)
#         v_est = pool.starmap(clus_velocity_calc, clus_param)

#         df_clusters_est_err = df_clus_bin.copy()
        
        

#         df_clusters_est_err['vx_est[km/s]'] = [i[0] for i in v_est]
#         df_clusters_est_err['vy_est[km/s]'] = [i[1] for i in v_est]
#         df_clusters_est_err['vz_est[km/s]'] = [i[2] for i in v_est]

#         df_clusters_est_err['vx_est_err[km/s]'] = [i[3] for i in v_est]
#         df_clusters_est_err['vy_est_err[km/s]'] = [i[4] for i in v_est]
#         df_clusters_est_err['vz_est_err[km/s]'] = [i[5] for i in v_est]
        
        
#         df_clusters_est_err['r_mag[kpc/h]'] = np.sqrt(df_clusters_est_err['x[kpc/h]']**2 +
#                                                       df_clusters_est_err['y[kpc/h]']**2 +
#                                                       df_clusters_est_err['z[kpc/h]']**2)
        
        
#         df_clusters_est_err['v_los[km/s]'] = (((df_clusters_est_err['vx[km/s]'] * df_clusters_est_err['x[kpc/h]']) +
#                                                (df_clusters_est_err['vy[km/s]'] * df_clusters_est_err['y[kpc/h]']) +
#                                                (df_clusters_est_err['vz[km/s]'] * df_clusters_est_err['z[kpc/h]'])) /
#                                                 df_clusters_est_err['r_mag[kpc/h]'])
        
#         df_clusters_est_err['v_los_est[km/s]'] = (((df_clusters_est_err['vx_est[km/s]'] * df_clusters_est_err['x[kpc/h]']) +
#                                                    (df_clusters_est_err['vy_est[km/s]'] * df_clusters_est_err['y[kpc/h]']) +
#                                                    (df_clusters_est_err['vz_est[km/s]'] * df_clusters_est_err['z[kpc/h]'])) /
#                                                     df_clusters_est_err['r_mag[kpc/h]'])
        
#         df_clusters_est_err['v_los_est_err[km/s]'] = np.sqrt((df_clusters_est_err['vx_est_err[km/s]'] * df_clusters_est_err['x[kpc/h]'] / df_clusters_est_err['r_mag[kpc/h]'])**2 +
#                                                              (df_clusters_est_err['vy_est_err[km/s]'] * df_clusters_est_err['y[kpc/h]'] / df_clusters_est_err['r_mag[kpc/h]'])**2 +
#                                                              (df_clusters_est_err['vz_est_err[km/s]'] * df_clusters_est_err['z[kpc/h]'] / df_clusters_est_err['r_mag[kpc/h]'])**2)

        
# #         linmix_slope, linmix_intercept, linmix_scatter = plotting_func(df_clusters_est_err, cell_size, clus_cube_size_actual)
        
# #         clear_output(wait=True)
        
                
# #         param_los_lmfit = lmfit_func(df_clusters_est_err['v_los[km/s]'], 
# #                                      df_clusters_est_err['v_los_est[km/s]']
# #                                    , df_clusters_est_err['v_los_est_err[km/s]']
# #                                   )
        
#         param_los_scipy = scipy_curve_fit_func(df_clusters_est_err['v_los[km/s]'], 
#                                      df_clusters_est_err['v_los_est[km/s]']
#                                    , df_clusters_est_err['v_los_est_err[km/s]'])




        
#         clus_mass_bins_results = clus_mass_bins_results.append({'Cell Size': cell_size,
#                 'Act Cube Size': clus_cube_size_actual,
#                 'Cube Size Set': clus_cube_size,
#                 'Clusters': no_of_clus,
                                                                
#                 'Lower lim': clus_min_mass,
#                 'Upper lim': clus_max_mass,                                                        
                                                                                 
#                 'Slope - Vlos': round(param_los_scipy[0][0], 3),  
#                 'Error - Slope - Vlos': round(np.sqrt(param_los_scipy[1][0, 0]), 3), 
#                 'Intercept - Vlos': round(param_los_scipy[0][1], 3),
#                 'Scatter - Vlos': 'Not-calculated',
               
#                                                                                }, 
#                 ignore_index=True)
        
        
        print('Time taken:', time.time() - t)
        print('\n')
        

10 250
Available cluster identified


Delta_gal: 0.05089513467293716


Time taken: 0.363431453704834


CPU times: user 23.5 ms, sys: 985 µs, total: 24.5 ms
Wall time: 364 ms
